##### Tugas

3. Buatlah model klasfikasi Multinomial Naive Bayes dengan ketentuan,
    1. Menggunakan data spam.csv
    2. Fitur TF-IDF dengan mengaktifkan stop_words
    3. Evaluasi hasilnya dan bandingkan dengan hasil pada Tugas no 2.
    4. Berikan kesimpulan fitur mana yang terbaik pada kasus data spam.csv

##### Langkah pertama: Impor library dan read dataset

In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv('assets/spam.csv', encoding='latin-1') # spesifiksi encoding diperlukan karena data tidak menggunakan UTF-8

df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


##### Langkah kedua: drop kolom yang tidak diperlukan

In [2]:
# Drop 3 kolom terakhir dengan fungsi iloc
df = df.drop(df.iloc[:,2:], axis=1)

# Cek data
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


##### Langkah ketiga: Ubah nama kolom v1 dan v2 ke 'Labels' dan 'SMS'

In [3]:
# Data untuk rename kolom v1 dan v2
new_cols = {
    'v1': 'Labels',
    'v2': 'SMS'
}

# Rename nama kolom v1 dan v2
df = df.rename(columns=new_cols)

# cek data
df.head()

,Labels,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


##### Langkah keempat: Inspeksi Data

In [4]:
# Cek Jumlah Data Per Kelas
print(df['Labels'].value_counts())
print('\n')

# Cek Kelengkapan Data
print(df.info())
print('\n')

# Cek Statistik Deskriptif
print(df.describe())

ham     4825
spam     747
Name: Labels, dtype: int64


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Labels  5572 non-null   object
 1   SMS     5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB
None


       Labels                     SMS
count    5572                    5572
unique      2                    5169
top       ham  Sorry, I'll call later
freq     4825                      30


##### Langkah kelima: Encode label yang telah di-rename

In [6]:
# Data untuk label
new_labels = {
    'spam': 1,
    'ham': 0
}

# Encode label
df['Labels'] = df['Labels'].map(new_labels)

# Cek data
df.head()

,Labels,SMS
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


##### Langkah enam: Memisahkan Fitur dengan Label

In [7]:
X = df['SMS'].values
y = df['Labels'].values

##### Langkah tujuh: Ekstraksi fitur (TF-IDF) dengan mengaktifkan stop words 

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# Split data training dan testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)

# Inisiasi CountVectorizer
bow = TfidfVectorizer(stop_words='english') 

# Fitting dan transform X_train dengan CountVectorizer
X_train = bow.fit_transform(X_train)

# Transform X_test
X_test = bow.transform(X_test)

In [9]:
print(len(bow.get_feature_names_out()))
print(f'Dimensi data: {X_train.shape}')

7527
Dimensi data: (4457, 7527)


##### Langkah delapan: Training dan evaluasi

In [10]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Inisiasi MultinomialNB
mnb = MultinomialNB()

# Fit model
mnb.fit(X_train, y_train)

# Prediksi dengan data training
y_pred_train = mnb.predict(X_train)

# Evaluasi akurasi data training
acc_train = accuracy_score(y_train, y_pred_train)

# Prediksi dengan data test
y_pred_test = mnb.predict(X_test)

# Evaluasi akurasi data test
acc_test = accuracy_score(y_test, y_pred_test)

# Print hasil evaluasi
print(f'Hasil akurasi data train: {acc_train}')
print(f'Hasil akurasi data test: {acc_test}')

Hasil akurasi data train: 0.983396903746915
Hasil akurasi data test: 0.9739910313901345


#### Evaluasi Hasil Dengan Tugas Nomor Dua
Setelah dilakukan training dan evaluasi data, model dapat denagn baik memprediksi data train dan kemungkinan tidak mengalami overfitting juga. Apabila dilakukan komparasi hasil dengan tugas nomor dua, terjadi pengurangan skor akurasi prediksi walau tidak signifikan. Hal tersebut diakibatkan dalam melakukan ekstraksi fitur menggunakan TD-IDF, mesin akan mengukur frekensi kemunculan kata, akan tetapi tetap mempertimbangkan bobot IDF. IDF berarti mengukur seberapa unik suatu kata dalam korpus teks tertentu, sehingga mesin masih mempertimbangkan bobot setiap kata berdasarkan keunikannya (semantiknya). Berbeda dengan countvectorizer yang tidak mempertimbangkan bobot atau semantiknya. 

#### Fitur Yang Terbaik Pada Kasus Spam

Fitur terbaik yang dapat digunakan untuk kasus spam.csv adalah TF-IDF, sebab dalam penggunannya, mesin akan menyoroti kata-kata yang lebih berarti daripada kata-kata umum pada email (memiliki semantik). Sehingga Keakuratan prediksi model dapat lebih terjamin karena semua kata tidak bernilai sama alias memiliki bobotnya masing-masing. 